# Aligning Data to Darwin Core - Sampling Event with Measurement or Fact using Python
Mathew Biddle

January 19, 2021

# General information about this notebook
This notebook was created for the [IOOS DMAC Code Sprint](https://www.glos.us/code-sprint/) Biological Data Session
The data in this notebook were created specifically as an example and meant solely to be
illustrative of the process for aligning data to the biological data standard - [Darwin Core](https://dwc.tdwg.org/).
These data should not be considered actually occurrences of species and any measurements
are also contrived. This notebook is meant to provide a step by step process for taking
original data and aligning it to Darwin Core

This notebook is a python implementation of the R notebook [IOOS_DMAC_DataToDWC_Notebook_event.R](https://github.com/ioos/bio_data_guide/blob/master/Standardizing%20Marine%20Biological%20Data/datasets/example_script_with_fake_data/IOOS_DMAC_DataToDwC_Notebook_event.R)

First let's bring in the appropriate libraries to work with the tabular data files and generate the appropriate content for the DarwinCore requirements.

_Note: [pyworms](https://github.com/iobis/pyworms) was installed using `pip install git+git://github.com/iobis/pyworms.git` to get the most recent version from GitHub._

In [1]:
import pandas as pd
import pyworms
import numpy as np
import uuid
import csv

Now we need to read in the raw data file using [pandas.read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).

In [2]:
file = 'data/MadeUpDataForBiologicalDataTraining.csv'
df = pd.read_csv(file, header=[0])

First we need to to decide if we will provide an **occurrence only** version of the data or a **sampling event with measurement or facts** version of the data. 

* **Occurrence** only is easier to create. It's only one file to produce. However, several pieces of information will be left out if we choose that option. 
* If we choose to do **sampling event with measurement or fact** we'll be able to capture all of the data in the file creating a lossless version.

Here we decide to use the **sampling event** option to include as much information as we can.

First let's create the `eventID` and `occurrenceID` in the original file so that information can be reused for all necessary files down the line.

In [3]:
df['eventID'] = df[['region', 'station', 'transect']].apply(lambda x: '_'.join(x.astype(str)), axis=1)
df['occurrenceID'] = uuid.uuid4()

We will need to create *three* separate files to comply with the **sampling event** format.
We'll start with the **event file** but we only need to include the columns that are relevant
to the event file.

# Event file

Let's first make a copy of the dataFrame we pulled in. Only using the data fields of interest for the **event file**.

In [4]:
event = df[['date', 'lat', 'lon', 'region', 'station', 'transect', 'depth', 'bottom type', 'eventID']].copy()

Next we need to rename any columns of data to match directly to Darwin Core.

In [5]:
event['decimalLatitude'] = event['lat']
event['decimalLongitude'] = event['lon']
event['minimumDepthInMeters'] = event['depth']
event['maximumDepthInMeters'] = event['depth']
event['habitat'] = event['bottom type']
event['island'] = event['region']

We need to convert the date to ISO format

In [7]:
event['eventDate'] = pd.to_datetime(
                            event['date'],
                            format='%m/%d/%Y',
                            utc=True)

We will also have to add any missing required fields

In [8]:
event['basisOfRecord'] = 'HumanObservation'
event['geodeticDatum'] = 'EPSG:4326 WGS84'

Then we'll remove any columns that we no longer need to clean things up a bit.

In [9]:
event.drop(
    columns=['date', 'lat', 'lon', 'region', 'station', 'transect', 'depth', 'bottom type'],
    inplace=True)

We have too many repeating rows of information. We can pare this down using eventID which
is a unique identifier for each sampling event in the data.

In [1]:
event.drop_duplicates(
    subset='eventID',
    inplace=True)

NameError: name 'event' is not defined

Finally we write out the event file

In [11]:
#event.to_csv(
#    'MadeUpData_event_frompy.csv',
#    header=True,
#    index=False,
#    date_format='%Y-%m-%d')

# Occurrence file
Next we need to create the occurrence file. We start by creating the dataframe.

In [12]:
occurrence = df[['scientific name', 'eventID', 'occurrenceID', 'percent cover']].copy()

Then we'll rename the columns that align directly with Darwin Core.

In [13]:
occurrence['scientificName'] = occurrence['scientific name']

Finally we'll add required information that's missing.

In [14]:
occurrence['occurrenceStatus'] = np.where(occurrence['percent cover'] == 0, 'absent', 'present')

## Taxonomic Name Matching
A requirement for OBIS is that all scientific names match to the World Register of
Marine Species (WoRMS) and a scientificNameID is included. A scientificNameID looks
like this "urn:lsid:marinespecies.org:taxname:275730" with the last digits after
the colon being the WoRMS aphia ID. We'll need to go out to WoRMS to grab this
information.

Create a lookup table of unique scientific names

In [15]:
lut_worms = pd.DataFrame(
    columns=['scientificName'],
    data=occurrence['scientificName'].unique())

Add the columns that we can grab information from WoRMS including the required scientificNameID.

In [16]:
headers = ['acceptedname', 'acceptedID', 'scientificNameID', 'kingdom', 'phylum',
           'class', 'order', 'family', 'genus', 'scientificNameAuthorship', 'taxonRank']

for head in headers:
    lut_worms[head] = ''

Taxonomic lookup using the library [pyworms](https://github.com/iobis/pyworms)

In [17]:
for index, row in lut_worms.iterrows():
    print('Searching for scientific name = %s' % row['scientificName'])
    resp = pyworms.aphiaRecordsByMatchNames(row['scientificName'])[0][0]
    lut_worms.loc[index, 'acceptedname'] = resp['valid_name']
    lut_worms.loc[index, 'acceptedID'] = resp['valid_AphiaID']
    lut_worms.loc[index, 'scientificNameID'] = resp['lsid']
    lut_worms.loc[index, 'kingdom'] = resp['kingdom']
    lut_worms.loc[index, 'phylum'] = resp['phylum']
    lut_worms.loc[index, 'class'] = resp['class']
    lut_worms.loc[index, 'order'] = resp['order']
    lut_worms.loc[index, 'family'] = resp['family']
    lut_worms.loc[index, 'genus'] = resp['genus']
    lut_worms.loc[index, 'scientificNameAuthorship'] = resp['authority']
    lut_worms.loc[index, 'taxonRank'] = resp['rank']

Searching for scientific name = Acropora cervicornis
Searching for scientific name = Madracis auretenra
Searching for scientific name = Mussa angulosa
Searching for scientific name = Siderastrea radians


Merge the lookup table of unique scientific names back with the occurrence data.

In [18]:
occurrence = pd.merge(occurrence, lut_worms, how='left', on='scientificName')

We're going to remove any unnecessary columns to clean up the file

In [19]:
occurrence.drop(
    columns=['scientific name', 'percent cover'],
    inplace=True)

Write out the file.

In [21]:
# sort the columns on scientificName
occurrence.sort_values('scientificName', inplace=True)

# reorganize column order to be consistent with R example:
columns = ["scientificName","eventID","occurrenceID","occurrenceStatus","acceptedname","acceptedID",
           "scientificNameID","kingdom","phylum","class","order","family","genus","scientificNameAuthorship",
           "taxonRank"]

#occurrence.to_csv(
#    "MadeUpData_Occurrence_frompy.csv",
#    header=True,
#    index=False,
#    quoting=csv.QUOTE_ALL,
#    columns=columns)

 All done with occurrence!

# Measurement Or Fact
The last file we need to create is the measurement or fact file. For this we need to
combine all of the measurements or facts that we want to include making sure to include
IDs from the BODC NERC vocabulary where possible.

In [22]:
temperature = df[['eventID', 'temperature', 'date']].copy()
temperature['occurrenceID'] = ''
temperature['measurementType'] = 'temperature'
temperature['measurementTypeID'] = 'http://vocab.nerc.ac.uk/collection/P25/current/WTEMP/'
temperature['measurementValue'] = temperature['temperature']
temperature['measurementUnit'] = 'Celsius'
temperature['measurementUnitID'] = 'http://vocab.nerc.ac.uk/collection/P06/current/UPAA/'
temperature['measurementAccuracy'] = 3
temperature['measurementDeterminedDate'] = pd.to_datetime(temperature['date'],
                                                          format='%m/%d/%Y',
                                                          utc=True)
temperature['measurementMethod'] = ''
temperature.drop(
    columns=['temperature', 'date'],
    inplace=True)

rugosity = df[['eventID', 'rugosity', 'date']].copy()
rugosity['occurrenceID'] = ''
rugosity['measurementType'] = 'rugosity'
rugosity['measurementTypeID'] = ''
rugosity['measurementValue'] = rugosity['rugosity'].map('{:,.6f}'.format)
rugosity['measurementUnit'] = ''
rugosity['measurementUnitID'] = ''
rugosity['measurementAccuracy'] = ''
rugosity['measurementDeterminedDate'] = pd.to_datetime(rugosity['date'],
                                                       format='%m/%d/%Y',
                                                       utc=True)
rugosity['measurementMethod'] = ''
rugosity.drop(
    columns=['rugosity', 'date'],
    inplace=True)

percent_cover = df[['eventID', 'occurrenceID', 'percent cover', 'date']].copy()
percent_cover['measurementType'] = 'Percent Cover'
percent_cover['measurementTypeID'] = 'http://vocab.nerc.ac.uk/collection/P01/current/SDBIOL10/'
percent_cover['measurementValue'] = percent_cover['percent cover']
percent_cover['measurementUnit'] = 'Percent/100m^2'
percent_cover['measurementUnitID'] = ''
percent_cover['measurementAccuracy'] = 5
percent_cover['measurementDeterminedDate'] = pd.to_datetime(percent_cover['date'],
                                                          format='%m/%d/%Y',
                                                          utc=True)
percent_cover['measurementMethod'] = ''
percent_cover.drop(
    columns=['percent cover', 'date'],
    inplace=True)

Concatenate all measurements or facts together into one dataFrame.

In [25]:
measurementorfact = pd.concat([temperature, rugosity, percent_cover])

Write measurement or fact file.

In [27]:
# measurementorfact.to_csv('MadeUpDate_mof_frompy.csv',
#                          index=False,
#                          header=True,
#                          date_format='%Y-%m-%d')